In [ ]:
!pip install rarfile

In [ ]:
import pandas as pd

In [ ]:
import rarfile
rf = rarfile.RarFile('/content/data.rar')
rf.extractall('data/')

In [ ]:
import os

for root, dirs, files in os.walk("/content/data"):
    for name in files:
        print(os.path.join(root, name))

/content/data/2024-09/2024-09-23_05-30-00.yaml
/content/data/2024-09/2024-09-17_05-30-00.yaml
/content/data/2024-09/2024-09-02_05-30-00.yaml
/content/data/2024-09/2024-09-11_05-30-00.yaml
/content/data/2024-09/2024-09-04_05-30-00.yaml
/content/data/2024-09/2024-09-20_05-30-00.yaml
/content/data/2024-09/2024-09-24_05-30-00.yaml
/content/data/2024-09/2024-09-27_05-30-00.yaml
/content/data/2024-09/2024-09-25_05-30-00.yaml
/content/data/2024-09/2024-09-18_05-30-00.yaml
/content/data/2024-09/2024-09-05_05-30-00.yaml
/content/data/2024-09/2024-09-26_05-30-00.yaml
/content/data/2024-09/2024-09-19_05-30-00.yaml
/content/data/2024-09/2024-09-10_05-30-00.yaml
/content/data/2024-09/2024-09-03_05-30-00.yaml
/content/data/2024-09/2024-09-30_05-30-00.yaml
/content/data/2024-09/2024-09-09_05-30-00.yaml
/content/data/2024-09/2024-09-12_05-30-00.yaml
/content/data/2024-09/2024-09-16_05-30-00.yaml
/content/data/2024-09/2024-09-13_05-30-00.yaml
/content/data/2024-09/2024-09-06_05-30-00.yaml
/content/data

In [ ]:
import yaml
with open('/content/data/2024-04/2024-04-24_05-30-00.yaml', 'r') as f:
    content = yaml.safe_load(f)
    print(content)

[{'Ticker': 'SBIN', 'close': 773.1, 'date': '2024-04-24 05:30:00', 'high': 778.5, 'low': 770.2, 'month': '2024-04', 'open': 778.5, 'volume': 7842615}, {'Ticker': 'BAJFINANCE', 'close': 7329.15, 'date': '2024-04-24 05:30:00', 'high': 7419.45, 'low': 7250.0, 'month': '2024-04', 'open': 7283.0, 'volume': 886929}, {'Ticker': 'TITAN', 'close': 3609.75, 'date': '2024-04-24 05:30:00', 'high': 3644.85, 'low': 3580.05, 'month': '2024-04', 'open': 3642.0, 'volume': 652811}, {'Ticker': 'ITC', 'close': 428.9, 'date': '2024-04-24 05:30:00', 'high': 430.8, 'low': 428.3, 'month': '2024-04', 'open': 429.25, 'volume': 5845170}, {'Ticker': 'TCS', 'close': 3831.05, 'date': '2024-04-24 05:30:00', 'high': 3898.0, 'low': 3824.45, 'month': '2024-04', 'open': 3890.0, 'volume': 1317312}, {'Ticker': 'LT', 'close': 3634.85, 'date': '2024-04-24 05:30:00', 'high': 3654.95, 'low': 3601.1, 'month': '2024-04', 'open': 3616.55, 'volume': 1501234}, {'Ticker': 'TATACONSUM', 'close': 1096.52, 'date': '2024-04-24 05:30:00

In [ ]:
import os, yaml, pandas as pd

base_dir = 'data/'  # after extraction
symbol_data = {}

for month in os.listdir(base_dir):
    month_path = os.path.join(base_dir, month)
    for file in os.listdir(month_path):
        if file.endswith('.yaml'):
            with open(os.path.join(month_path, file), 'r') as f:
                content = yaml.safe_load(f)
                for record in content:
                    symbol = record['Ticker']
                    df = pd.DataFrame([record])
                    symbol_data.setdefault(symbol, []).append(df)

# Save 50 CSVs (one for each symbol)
os.makedirs('csv_output', exist_ok=True)
for symbol, dfs in symbol_data.items():
    full_df = pd.concat(dfs)
    full_df.to_csv(f'csv_output/{symbol}.csv', index=False)

In [ ]:
import glob

all_files = glob.glob("csv_output/*.csv")
combined_df = pd.concat([pd.read_csv(f) for f in all_files])
combined_df.to_csv("combined_stocks.csv", index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('combined_stocks.csv')
df['date'] = pd.to_datetime(df['date'])  # Ensure datetime format
df = df.sort_values(['Ticker', 'date'])  # Important for calculations

In [ ]:
df['daily_return'] = df.groupby('Ticker')['close'].pct_change()

In [ ]:
df['daily_return'] = df.groupby('Ticker')['close'].pct_change()
df['cumulative_return'] = df.groupby('Ticker')['daily_return'].transform(lambda x: (1 + x).cumprod())


In [ ]:
volatility_df = df.groupby('Ticker')['daily_return'].std().reset_index()
volatility_df.columns = ['Ticker', 'volatility']

In [ ]:
df['month'] = df['date'].dt.to_period('M')
monthly_return = df.groupby(['Ticker', 'month'])['daily_return'].sum().reset_index()

In [ ]:
df.to_csv("engineered_stocks.csv", index=False)
volatility_df.to_csv("volatility.csv", index=False)
monthly_return.to_csv("monthly_return.csv", index=False)

In [ ]:
sector_mapping = {
    'RELIANCE': 'Energy',
    'TCS': 'IT',
    'INFY': 'IT',
    'HDFCBANK': 'Financials',
    'ICICIBANK': 'Financials',
    'BAJAJ-AUTO': 'Auto',
    'SBIN': 'Financials',
    'NTPC': 'Energy',
    'TITAN': 'Consumer Goods',
    'MARUTI': 'Auto',
    # 🔧 Add all remaining tickers as per your dataset
}

df['Sector'] = df['Ticker'].map(sector_mapping)

In [ ]:
# Ensure cumulative_return is already computed per Ticker (which it is in your data)
sector_performance = df.groupby('Sector')['cumulative_return'].mean().sort_values(ascending=False).reset_index()
sector_performance.columns = ['Sector', 'Avg_Yearly_Return']